In [1]:
import matplotlib.pyplot as plt
import numpy as np
# import networkx as nx
import os
import time
import pickle
import json
import pandas as pd
from numba import jit
from types import SimpleNamespace
# from power_planner.data_reader import DataReader
from power_planner import graphs
from power_planner.utils.utils import (compute_pylon_dists)

In [2]:
PATH_FILES = "../data"

# DEFINE CONFIGURATION
ID = "w_ksp_5"  # str(round(time.time() / 60))[-5:]

OUT_PATH = "outputs/path_" + ID
SCALE_PARAM = 5  # args.scale
# normal graph pipeline
# PIPELINE = [(2, 50), (1, 0)]  # [(1, 0)]  # [(4, 80), (2, 50), (1, 0)]  #
# random graph pipeline
PIPELINE = [(1, 0)]  # [(0.9, 40), (0, 0)]

GRAPH_TYPE = graphs.LineGraph
# LineGraph, WeightedGraph, RandomWeightedGraph, RandomLineGraph, PowerBF
# TwoPowerBF, WeightedKSP
print("graph type:", GRAPH_TYPE)
# summarize: mean/max/min, remove: all/surrounding, sample: simple/watershed
NOTES = "None"  # "mean-all-simple"

IOPATH = os.path.join(PATH_FILES, "belgium_data_1_" + str(SCALE_PARAM) + ".dat")
    
# READ DATA
with open(IOPATH, "rb") as infile:
    data = pickle.load(infile)
    (instance, edge_inst, instance_corr, config) = data

graph type: <class 'power_planner.graphs.line_graph.LineGraph'>


In [3]:
graph = GRAPH_TYPE(instance, instance_corr)

In [4]:
path, path_costs, cost_sum = graph.single_sp(**vars(config.graph))

1) Initialize shifts and instance (corridor)
['angle', 'cultural', 'parcel', 'planning', 'technical'] [0.33333333 0.25       0.16666667 0.25      ]
['angle', 'cultural', 'parcel', 'planning', 'technical'] [0.09090909 0.3030303  0.22727273 0.15151515 0.22727273]
Added nodes: 1070160 in time: 0.446260929107666
2) Initialize distances to inf and predecessors
n_neighbors: 568
Done adding edges: 9590393
DONE adding 9590393 edges: 20.302155017852783
3) Compute source shortest path tree
number of vertices and edges: 1070160 9590393
4) shortest path 10.287088832186917


In [ ]:
graph.graph.get_shortest_path(graph.start_inds, graph.dest_inds)

In [ ]:
_ = graph.sp_trees(edge_inst, **vars(config.graph))

In [ ]:
from power_planner.alternative_paths import AlternativePaths
from power_planner.ksp import KSP

In [ ]:
ksp = KSP(graph)
out_ksp = ksp.laplace(5)

In [ ]:
alt = AlternativePaths(graph)
window_path, _, _ = alt.replace_window(150, 200,250, 300)

In [ ]:
window_path = np.asarray(window_path)
path_orig = np.asarray(out_ksp[0][0])
plt.plot(window_path[:,0], window_path[:,1])
plt.plot(path_orig[:,0], path_orig[:,1])

In [7]:
config.graph

namespace(ANGLE_WEIGHT=0.1, CSV_TIMES='../outputs/time_tests.csv', EDGE_WEIGHT=0.33, GTNX=1, KSP=5, MAX_ANGLE=1.57079632679489, MAX_ANGLE_LG=1.57079632679489, PYLON_DIST_MAX=5.0, PYLON_DIST_MIN=3.0, VERBOSE=1, class_weights=[4.0, 3.0, 2.0, 3.0], dest_inds=array([ 39, 238]), layer_classes=['cultural', 'parcel', 'planning', 'technical'], orig_dest=array([ 471, 1202]), orig_start=array([816, 124]), scale=5, start_inds=array([108,  22]), transform_matrix=Affine(10.0, 0.0, 163246.20677687135,
       0.0, -10.0, 139261.12138460297))

### Evaluation of outputs

In [ ]:
with open("../../outputs/power analysis/power_analysis_belgium_paths.dat", "rb") as infile:
    #power analysis/power_analysis_belgium_paths.dat
    paths = pickle.load(infile)

In [ ]:
import seaborn as sb

In [ ]:
for p in paths:
    print("mean:", round(np.mean(p), 2), "sum:", round(np.sum(p), 2))
    sb.distplot(p)
    plt.ylim(0,0.4)
    plt.show()

## Create random instance for Github

In [ ]:
test = instance_corr.copy()
test[6:245, :60]=1
plt.imshow(test)

In [ ]:
[1] + [round(c,1) for c in np.logspace(0.1, 0.6, 6)]

In [ ]:
random_inst = np.random.rand(*instance.shape)

In [ ]:
random_inst.shape

In [ ]:
random_edge_inst = random_inst + (np.random.rand(*instance.shape)-0.5)*0.1

In [ ]:
random_inst[2,3,3]

In [ ]:
from scipy.ndimage import gaussian_filter

In [ ]:
smooth = np.asarray([gaussian_filter(random_inst[i], 5) for i in range(3)])

In [ ]:
plt.imshow(smooth[0])
plt.colorbar()

In [ ]:
smooth_edge = np.asarray([gaussian_filter(random_edge_inst[i], 5) for i in range(3)])

In [ ]:
smooth = (smooth-np.min(smooth))/(np.max(smooth)-np.min(smooth))

In [ ]:
smooth_edge = (smooth_edge-np.min(smooth_edge))/(np.max(smooth_edge)-np.min(smooth_edge))

In [ ]:
data_out = (smooth, smooth_edge, test, config)
with open("../data/test_data_1_2.dat", "wb") as outfile:
    pickle.dump(data_out, outfile)
print("successfully saved data")

In [ ]:
config.graph.scale

### Check average cost path - did running average work?

In [ ]:
nonavg = pd.read_csv("../../outputs/test_nonavg_ch_coords.csv_0.csv")
avg = pd.read_csv("../../outputs/test_avg_ch_coords.csv_0.csv")

In [ ]:
avg_path = (np.asarray(avg[["X_raw", "Y_raw"]])/5).astype(int)
nonavg_path = (np.asarray(nonavg[["X_raw", "Y_raw"]])/5).astype(int)

In [ ]:
costs1 = [graph.instance[tuple(p)] for p in avg_path]
costs2 = [graph.instance[tuple(p)] for p in nonavg_path]

In [ ]:
np.sum(costs1), np.mean(costs1), np.sum(costs2), np.mean(costs2)